### 05. Training

We'll load the previously saved CNN+RNN model and train it here.

In [22]:
import pickle

from keras.models import model_from_json
from keras.models import load_model
from keras.preprocessing.sequence import pad_sequences
from keras.utils.np_utils import to_categorical
from numpy import array

In [20]:
# standard variables
MAX_LENGTH = 52
VOCAB_SIZE = 6321 # Using unique words as vocab here.
NPIX = 299
TARGET_SIZE = (NPIX,NPIX,3)
EMBEDDING_SIZE = 300

In [2]:
# data generator, intended to be used in a call to model.fit_generator()

def data_generator(descriptions, photos, MAX_LENGTH,VOCAB_SIZE, num_photos_per_batch):
    X1, X2, y = list(), list(), list()
    n=0
    # loop for ever over images
    while 1:
        for key, desc_list in train_descriptions.items():
            n+=1
            # retrieve the photo feature
            photo = photos[key]
            for desc in desc_list:
                seq = token.texts_to_sequences([desc])
                seq = seq[0]
                for i in range(1,len(seq)):
                    in_seq , op_seq = seq[:i],seq[i]
                    #converting input sequence to fix length
                    in_seq = pad_sequences([in_seq],maxlen=MAX_LENGTH,padding="post")[0]
                    # converting op_seq to vocabulary size
                    op_seq = to_categorical([op_seq],num_classes=VOCAB_SIZE)[0]
                    X1.append(photo)
                    X2.append(in_seq)
                    y.append(op_seq)
            # yield the batch data
            if n==num_photos_per_batch:
                yield [[array(X1), array(X2)], array(y)]
                X1, X2, y = list(), list(), list()
                n=0

In [6]:
def load_pickle(path):
    with open(path,"rb") as f:
        obj = pickle.load(f)
    return obj

In [7]:
train_image_extracted = load_pickle("train_image_extracted.pkl")

In [8]:
train_descriptions = load_pickle("corruption_free_coco_descriptions.pkl")

In [9]:
token = load_pickle("token.pkl")

In [10]:
def load_model(path):
    with open(path,"r") as f:
        model = model_from_json(f.read())
    print("Model loaded successfully")
    return model

In [11]:
model = load_model("model.json")

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Model loaded successfully


In [12]:
# loading the weights of the model
model.load_weights("model.h5")

In [13]:
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [ ]:
model.summary()

Total Epochs = 30

For 1st 20 epoch , batch size = 5 images ====> stpes per epoch = 1200

In [16]:
epochs = 20
batch_size = number_pics_per_batch = 5
steps = len(train_descriptions)//number_pics_per_batch
steps

16556

In [15]:
len(train_descriptions)

82782

In [ ]:
for i in range(epochs):
#     if epochs<11:
    generator = data_generator(train_descriptions,train_image_extracted,MAX_LENGTH,VOCAB_SIZE,number_pics_per_batch)
    model.fit_generator(generator, epochs=1, steps_per_epoch=steps, verbose=1)
    model.save('model_weights/model_demo' + str(i) + '.h5')

#### modifying learning rate to 0.0001

[Link for More Info](https://towardsdatascience.com/image-captioning-with-keras-teaching-computers-to-describe-pictures-c88a46a311b8)

In [ ]:
batch_size = number_pics_per_batch = 10
steps = len(train_descriptions)//number_pics_per_batch
epochs = 10

model.optimizer.lr = 0.0001

In [ ]:
for i in range(epochs):
    generator = data_generator(train_descriptions,train_image_extracted,MAX_LENGTH,VOCAB_SIZE,number_pics_per_batch)
    model.fit_generator(generator, epochs=1, steps_per_epoch=steps, verbose=1)

In [ ]:
model.save_weights('model_weights/model_final.h5')